In [30]:
import pandas as pd

In [31]:
maps = pd.read_csv('clean_data/SF_block_group_shapes.csv', index_col=0, dtype={'full_fips':str})
maps.head()

,objectid,full_fips,Shape__Area,Shape__Length
0,1966,060750101001,0.000123,0.053583
1,1967,060750101002,0.000022,0.028353
2,1968,060750102001,0.000057,0.038006
3,1969,060750102002,0.000013,0.019144
4,1970,060750102003,0.000013,0.019144


# First: no rates, make all evictions spatial

In [32]:
all_evictions = pd.read_csv('clean_data/all_evictions_agg_counts.csv', dtype={'block':str})
all_evictions.head()

,block,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,Demolition,...,Substantial Rehab,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions
0,060750101001,11,54,27,2,22,1,4,0,0,...,0,0,0,0,3,3,0,0,0,109
1,060750101002,7,53,25,3,0,0,0,19,10,...,1,23,0,3,4,5,0,0,0,157
2,060750102001,9,34,16,2,0,0,2,21,1,...,0,12,3,3,3,5,0,0,0,118
3,060750102002,12,24,19,4,0,0,1,27,1,...,0,16,0,0,4,13,0,0,0,122
4,060750102003,1,2,8,2,0,0,3,10,2,...,0,0,2,0,2,2,0,0,0,34


In [33]:
print(len(maps) - len(all_evictions))
len(maps), len(all_evictions)

4


(581, 577)

In [34]:
diff = list(set(maps.full_fips) - set(all_evictions.block))

In [35]:
maps[maps.full_fips.isin(diff)]

,objectid,full_fips,Shape__Area,Shape__Length
552,2518,060750605021,0.000007,0.013761
553,2519,060750605022,0.000008,0.014248
576,2542,060759804011,0.025340,0.654465
580,2546,060759901000,0.007253,0.378299


In [36]:
all_evictions.tail()

,block,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,Demolition,...,Substantial Rehab,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions
572,060759802001,1,2,3,0,0,0,1,10,0,...,0,5,0,0,0,1,0,0,0,25
573,060759803001,12,30,26,3,0,1,3,35,0,...,0,15,1,7,5,7,0,0,0,153
574,060759805011,8,2,1,0,0,1,0,4,0,...,0,0,0,1,1,0,0,0,0,18
575,060759806001,0,0,1,0,0,0,0,1,0,...,0,0,0,1,2,0,0,0,0,5
576,060759809001,4,2,6,1,0,1,0,1,0,...,0,0,0,2,3,2,0,0,0,18


In [37]:
for fips in diff:
    all_evictions = all_evictions.append(pd.Series(0, index=all_evictions.columns), ignore_index=True)
    all_evictions.loc[len(all_evictions)-1, 'block'] = str(fips)
all_evictions.tail()

,block,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,Demolition,...,Substantial Rehab,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions
576,060759809001,4,2,6,1,0,1,0,1,0,...,0,0,0,2,3,2,0,0,0,18
577,060750605022,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
578,060750605021,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
579,060759901000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
580,060759804011,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
print(len(maps) - len(all_evictions))
len(maps), len(all_evictions)

0


(581, 581)

In [39]:
# Joining should change nothing about length regardless of join type
joint_all = pd.merge(left=all_evictions, right=maps, left_on='block', right_on='full_fips', how='left')
joint_all.head()

,block,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,Demolition,...,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions,objectid,full_fips,Shape__Area,Shape__Length
0,060750101001,11,54,27,2,22,1,4,0,0,...,3,3,0,0,0,109,1966,060750101001,0.000123,0.053583
1,060750101002,7,53,25,3,0,0,0,19,10,...,4,5,0,0,0,157,1967,060750101002,0.000022,0.028353
2,060750102001,9,34,16,2,0,0,2,21,1,...,3,5,0,0,0,118,1968,060750102001,0.000057,0.038006
3,060750102002,12,24,19,4,0,0,1,27,1,...,4,13,0,0,0,122,1969,060750102002,0.000013,0.019144
4,060750102003,1,2,8,2,0,0,3,10,2,...,2,2,0,0,0,34,1970,060750102003,0.000013,0.019144


In [40]:
len(joint_all)

581

In [41]:
joint_all.drop(columns='full_fips').to_csv('clean_data/mappable_evictions.csv')

# Second: Change 2018 Evictions To Rates

In [42]:
ev18 = pd.read_csv('clean_data/2018_eviction_agg_counts.csv', dtype={'block':str})
ev18.head()

,block,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,Demolition,...,Substantial Rehab,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions
0,060750101001,0,2,1,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
1,060750101002,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,060750102001,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,060750102002,1,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
4,060750102003,0,0,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,3


In [43]:
acs18 = pd.read_csv('clean_data/2018_acs_housing.csv', index_col=0, dtype={'geoid':str})
acs18.head()

,geoid,total
0,060750101001,897
1,060750101002,2953
2,060750102001,1643
3,060750102002,1764
4,060750102003,892


In [44]:
print(len(acs18) - len(ev18))
len(acs18), len(ev18)

169


(581, 412)

In [45]:
diff = list(set(acs18.geoid) - set(ev18.block))
len(diff)
acs18[acs18.geoid.isin(diff)]

,geoid,total
12,060750105001,963
13,060750105002,1969
19,060750107003,1718
20,060750107004,1618
34,060750112002,1356
...,...,...
576,060759804011,0
577,060759805011,669
578,060759806001,575
579,060759809001,272


In [46]:
acs18[acs18.total == 0]

,geoid,total
172,060750176011,0
576,060759804011,0
580,060759901000,0


In [47]:
joint18 = pd.merge(left=acs18, right=ev18, left_on='geoid', right_on='block', how='left')
joint18.tail()

,geoid,total,block,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,...,Substantial Rehab,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions
576,060759804011,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
577,060759805011,669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
578,060759806001,575,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
579,060759809001,272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
580,060759901000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
joint18.fillna(0, inplace=True)
joint18.tail()

,geoid,total,block,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,...,Substantial Rehab,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions
576,060759804011,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
577,060759805011,669,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
578,060759806001,575,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
579,060759809001,272,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
580,060759901000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
joint18.drop(columns='block', inplace=True)
joint18.tail()

,geoid,total,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,...,Substantial Rehab,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions
576,060759804011,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
577,060759805011,669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
578,060759806001,575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
579,060759809001,272,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
580,060759901000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
joint18[joint18.total == 0]

,geoid,total,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,...,Substantial Rehab,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions
172,060750176011,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
576,060759804011,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
580,060759901000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now to convert counts into rates per 1000 households:
1. Convert total into units of 1000 households
2. Divide Relevent columns by total
3. Set NaN to 0 (caused by dividing by 0)

In [51]:
joint18['hh_1000'] = joint18.total / 1000
joint18.drop(columns='total', inplace=True)
joint18#.head(3)

,geoid,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,Demolition,...,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions,hh_1000
0,060750101001,0.0,2.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.897
1,060750101002,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.953
2,060750102001,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.643
3,060750102002,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.764
4,060750102003,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,060759804011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
577,060759805011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.669
578,060759806001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.575
579,060759809001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.272


In [52]:
# df[['B','C']] = df[['B','C']].div(df.A, axis=0)
joint18.iloc[:, 1:-1] = joint18.iloc[:, 1:-1].div(joint18.hh_1000, axis=0).fillna(0)
joint18

,geoid,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,Demolition,...,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions,hh_1000
0,060750101001,0.000000,2.229654,1.114827,0.0,3.344482,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.574136,0.897
1,060750101002,0.338639,0.338639,0.338639,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.015916,2.953
2,060750102001,0.608643,0.608643,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.217285,1.643
3,060750102002,0.566893,1.700680,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.834467,1.764
4,060750102003,0.000000,0.000000,2.242152,0.0,0.000000,0.0,1.121076,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.363229,0.892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,060759804011,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000
577,060759805011,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.669
578,060759806001,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.575
579,060759809001,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.272


In [53]:
joint18.sort_values('Total Evictions', ascending=False)

,geoid,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,Demolition,...,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions,hh_1000
575,060759803001,45.454545,0.000000,15.151515,0.000000,0.0,0.0,30.30303,15.151515,0.0,...,45.454545,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,151.515152,0.066
59,060750124022,0.000000,0.000000,0.000000,0.691563,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,15.905947,0.0,103.734440,1.446
469,060750332032,0.000000,38.232795,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,38.232795,1.177
472,060750332043,0.000000,32.676349,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,32.676349,1.928
468,060750332031,0.000000,22.900763,0.381679,0.000000,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,23.282443,2.620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,060750304002,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.171
387,060750305003,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.451
388,060750306001,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.902
389,060750306002,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.775


In [54]:
joint18.loc[59]

geoid                      060750124022
Non Payment                           0
Breach                                0
Nuisance                              0
Illegal Use                    0.691563
Failure to Sign Renewal               0
Access Denial                         0
Unapproved Subtenant                  0
Owner Move In                         0
Demolition                            0
Capital Improvement             87.1369
Substantial Rehab                     0
Ellis Act WithDrawal                  0
Condo Conversion                      0
Roommate Same Unit                    0
Other Cause                           0
Late Payments                         0
Lead Remediation                      0
Development                     15.9059
Good Samaritan Ends                   0
Total Evictions                 103.734
hh_1000                           1.446
Name: 59, dtype: object

In [55]:
joint18 = joint18.sort_values('Total Evictions', ascending=False).drop(index=575)
joint18

,geoid,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,Demolition,...,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions,hh_1000
59,060750124022,0.0,0.000000,0.000000,0.691563,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,15.905947,0.0,103.734440,1.446
469,060750332032,0.0,38.232795,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,38.232795,1.177
472,060750332043,0.0,32.676349,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,32.676349,1.928
468,060750332031,0.0,22.900763,0.381679,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,23.282443,2.620
470,060750332041,0.0,17.492711,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,17.492711,1.029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,060750304002,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.171
387,060750305003,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.451
388,060750306001,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.902
389,060750306002,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.775


In [56]:
joint18.set_index('geoid').to_csv('clean_data/2018_ev_rates.csv')

# Third: Change Cumulative Counts to Rates on 2019 Households

In [57]:
ev_cum = pd.read_csv('clean_data/cumulative_eviction_agg_counts.csv', dtype={'block':str})
ev_cum.head()

,block,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,Demolition,...,Substantial Rehab,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions
0,060750101001,11,51,22,2,16,1,4,0,0,...,0,0,0,0,2,3,0,0,0,97
1,060750101002,7,53,20,3,0,0,0,19,10,...,1,23,0,3,4,5,0,0,0,152
2,060750102001,8,32,14,2,0,0,2,21,1,...,0,12,3,3,3,3,0,0,0,111
3,060750102002,11,22,14,4,0,0,1,26,1,...,0,16,0,0,4,13,0,0,0,112
4,060750102003,1,2,6,2,0,0,3,10,2,...,0,0,2,0,2,2,0,0,0,32


In [58]:
acs19 = pd.read_csv('clean_data/2019_acs_housing.csv', index_col=0, dtype={'geoid':str})
acs19.head()

,geoid,total
0,060750101001,770
1,060750101002,3119
2,060750102001,1590
3,060750102002,1668
4,060750102003,909


In [59]:
print(len(acs19) - len(ev_cum))
len(acs19), len(ev_cum)

4


(581, 577)

In [60]:
diff = list(set(acs19.geoid) - set(ev_cum.block))
len(diff)
acs19[acs19.geoid.isin(diff)]

,geoid,total
552,060750605021,966
553,060750605022,794
576,060759804011,0
580,060759901000,0


In [61]:
acs19[acs19.total == 0]

,geoid,total
172,060750176011,0
576,060759804011,0
580,060759901000,0


In [62]:
joint19 = pd.merge(left=acs19, right=ev_cum, left_on='geoid', right_on='block', how='left')
joint19.tail()

,geoid,total,block,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,...,Substantial Rehab,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions
576,060759804011,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
577,060759805011,507,060759805011,8.0,2.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,17.0
578,060759806001,690,060759806001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,4.0
579,060759809001,253,060759809001,3.0,2.0,6.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,16.0
580,060759901000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
joint19.fillna(0, inplace=True)
joint19.tail()

,geoid,total,block,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,...,Substantial Rehab,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions
576,060759804011,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
577,060759805011,507,060759805011,8.0,2.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,17.0
578,060759806001,690,060759806001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,4.0
579,060759809001,253,060759809001,3.0,2.0,6.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,16.0
580,060759901000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
joint19.drop(columns='block', inplace=True)
joint19.tail()

,geoid,total,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,...,Substantial Rehab,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions
576,060759804011,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
577,060759805011,507,8.0,2.0,0.0,0.0,0.0,1.0,0.0,4.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,17.0
578,060759806001,690,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,4.0
579,060759809001,253,3.0,2.0,6.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,16.0
580,060759901000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
joint19.iloc[172, 1:] = 0
joint19.iloc[172]

geoid                      060750176011
total                                 0
Non Payment                           0
Breach                                0
Nuisance                              0
Illegal Use                           0
Failure to Sign Renewal               0
Access Denial                         0
Unapproved Subtenant                  0
Owner Move In                         0
Demolition                            0
Capital Improvement                   0
Substantial Rehab                     0
Ellis Act WithDrawal                  0
Condo Conversion                      0
Roommate Same Unit                    0
Other Cause                           0
Late Payments                         0
Lead Remediation                      0
Development                           0
Good Samaritan Ends                   0
Total Evictions                       0
Name: 172, dtype: object

In [66]:
joint19.loc[575]

geoid                      060759803001
total                                63
Non Payment                          12
Breach                               29
Nuisance                             26
Illegal Use                           3
Failure to Sign Renewal               0
Access Denial                         1
Unapproved Subtenant                  3
Owner Move In                        35
Demolition                            0
Capital Improvement                  12
Substantial Rehab                     0
Ellis Act WithDrawal                 15
Condo Conversion                      0
Roommate Same Unit                    7
Other Cause                           5
Late Payments                         7
Lead Remediation                      0
Development                           0
Good Samaritan Ends                   0
Total Evictions                     151
Name: 575, dtype: object

In [67]:
joint19[joint19.total == 0]

,geoid,total,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,...,Substantial Rehab,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions
172,060750176011,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
576,060759804011,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
580,060759901000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now to convert counts into rates per 1000 households:
1. Convert total into units of 1000 households
2. Divide Relevent columns by total
3. Set NaN to 0 (caused by dividing by 0)

In [68]:
joint19['hh_1000'] = joint19.total / 1000
joint19.drop(columns='total', inplace=True)
joint19#.head(3)

,geoid,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,Demolition,...,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions,hh_1000
0,060750101001,11.0,51.0,22.0,2.0,16.0,1.0,4.0,0.0,0.0,...,0.0,0.0,0.0,2.0,3.0,0.0,0.0,0.0,97.0,0.770
1,060750101002,7.0,53.0,20.0,3.0,0.0,0.0,0.0,19.0,10.0,...,23.0,0.0,3.0,4.0,5.0,0.0,0.0,0.0,152.0,3.119
2,060750102001,8.0,32.0,14.0,2.0,0.0,0.0,2.0,21.0,1.0,...,12.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,111.0,1.590
3,060750102002,11.0,22.0,14.0,4.0,0.0,0.0,1.0,26.0,1.0,...,16.0,0.0,0.0,4.0,13.0,0.0,0.0,0.0,112.0,1.668
4,060750102003,1.0,2.0,6.0,2.0,0.0,0.0,3.0,10.0,2.0,...,0.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,32.0,0.909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,060759804011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
577,060759805011,8.0,2.0,0.0,0.0,0.0,1.0,0.0,4.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,17.0,0.507
578,060759806001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,4.0,0.690
579,060759809001,3.0,2.0,6.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,16.0,0.253


In [69]:
# df[['B','C']] = df[['B','C']].div(df.A, axis=0)
joint19.iloc[:, 1:-1] = joint19.iloc[:, 1:-1].div(joint19.hh_1000, axis=0).fillna(0)
joint19

,geoid,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,Demolition,...,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions,hh_1000
0,060750101001,14.285714,66.233766,28.571429,2.597403,20.779221,1.298701,5.194805,0.000000,0.000000,...,0.000000,0.000000,0.000000,2.597403,3.896104,0.0,0.0,0.0,125.974026,0.770
1,060750101002,2.244309,16.992626,6.412312,0.961847,0.000000,0.000000,0.000000,6.091696,3.206156,...,7.374158,0.000000,0.961847,1.282462,1.603078,0.0,0.0,0.0,48.733568,3.119
2,060750102001,5.031447,20.125786,8.805031,1.257862,0.000000,0.000000,1.257862,13.207547,0.628931,...,7.547170,1.886792,1.886792,1.886792,1.886792,0.0,0.0,0.0,69.811321,1.590
3,060750102002,6.594724,13.189448,8.393285,2.398082,0.000000,0.000000,0.599520,15.587530,0.599520,...,9.592326,0.000000,0.000000,2.398082,7.793765,0.0,0.0,0.0,67.146283,1.668
4,060750102003,1.100110,2.200220,6.600660,2.200220,0.000000,0.000000,3.300330,11.001100,2.200220,...,0.000000,2.200220,0.000000,2.200220,2.200220,0.0,0.0,0.0,35.203520,0.909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,060759804011,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000
577,060759805011,15.779093,3.944773,0.000000,0.000000,0.000000,1.972387,0.000000,7.889546,0.000000,...,0.000000,0.000000,1.972387,1.972387,0.000000,0.0,0.0,0.0,33.530572,0.507
578,060759806001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.449275,0.000000,...,0.000000,0.000000,1.449275,2.898551,0.000000,0.0,0.0,0.0,5.797101,0.690
579,060759809001,11.857708,7.905138,23.715415,3.952569,0.000000,3.952569,0.000000,3.952569,0.000000,...,0.000000,0.000000,7.905138,7.905138,7.905138,0.0,0.0,0.0,63.241107,0.253


In [70]:
joint19.set_index('geoid')

,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,Demolition,Capital Improvement,...,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions,hh_1000
geoid,,,,,,,,,,,,,,,,,,,,,
060750101001,14.285714,66.233766,28.571429,2.597403,20.779221,1.298701,5.194805,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,2.597403,3.896104,0.0,0.0,0.0,125.974026,0.770
060750101002,2.244309,16.992626,6.412312,0.961847,0.000000,0.000000,0.000000,6.091696,3.206156,1.282462,...,7.374158,0.000000,0.961847,1.282462,1.603078,0.0,0.0,0.0,48.733568,3.119
060750102001,5.031447,20.125786,8.805031,1.257862,0.000000,0.000000,1.257862,13.207547,0.628931,6.289308,...,7.547170,1.886792,1.886792,1.886792,1.886792,0.0,0.0,0.0,69.811321,1.590
060750102002,6.594724,13.189448,8.393285,2.398082,0.000000,0.000000,0.599520,15.587530,0.599520,1.199041,...,9.592326,0.000000,0.000000,2.398082,7.793765,0.0,0.0,0.0,67.146283,1.668
060750102003,1.100110,2.200220,6.600660,2.200220,0.000000,0.000000,3.300330,11.001100,2.200220,2.200220,...,0.000000,2.200220,0.000000,2.200220,2.200220,0.0,0.0,0.0,35.203520,0.909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
060759804011,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000
060759805011,15.779093,3.944773,0.000000,0.000000,0.000000,1.972387,0.000000,7.889546,0.000000,0.000000,...,0.000000,0.000000,1.972387,1.972387,0.000000,0.0,0.0,0.0,33.530572,0.507
060759806001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.449275,0.000000,0.000000,...,0.000000,0.000000,1.449275,2.898551,0.000000,0.0,0.0,0.0,5.797101,0.690


In [71]:
joint19.sort_values('Total Evictions', ascending=False)

,geoid,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,Demolition,...,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions,hh_1000
575,060759803001,190.476190,460.317460,412.698413,47.619048,0.0,15.873016,47.619048,555.555556,0.000000,...,238.095238,0.0,111.111111,79.365079,111.111111,0.0,0.0,0.000000,2396.825397,0.063
469,060750332032,3.381234,277.261200,23.668639,0.000000,0.0,0.000000,11.834320,0.000000,0.000000,...,0.000000,0.0,0.000000,0.845309,0.845309,0.0,0.0,2.535926,430.262046,1.183
472,060750332043,0.000000,144.694534,11.254019,0.000000,0.0,0.000000,8.038585,0.000000,0.000000,...,0.000000,0.0,0.535906,0.000000,2.143623,0.0,0.0,0.000000,224.008574,1.866
62,060750125012,1.631321,110.114192,105.220228,1.631321,0.0,0.815661,1.631321,0.815661,0.000000,...,0.815661,0.0,0.000000,1.631321,0.815661,0.0,0.0,0.000000,212.887439,1.226
269,060750229032,18.720749,34.321373,35.881435,1.560062,0.0,0.000000,6.240250,43.681747,4.680187,...,28.081123,0.0,6.240250,6.240250,6.240250,0.0,0.0,0.000000,188.767551,0.641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,060759804011,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000
172,060750176011,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000
552,060750605021,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.966
553,060750605022,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.794


In [72]:
joint19 = joint19.sort_values('Total Evictions', ascending=False).drop(index=575)
joint19

,geoid,Non Payment,Breach,Nuisance,Illegal Use,Failure to Sign Renewal,Access Denial,Unapproved Subtenant,Owner Move In,Demolition,...,Ellis Act WithDrawal,Condo Conversion,Roommate Same Unit,Other Cause,Late Payments,Lead Remediation,Development,Good Samaritan Ends,Total Evictions,hh_1000
469,060750332032,3.381234,277.261200,23.668639,0.000000,0.0,0.000000,11.834320,0.000000,0.000000,...,0.000000,0.0,0.000000,0.845309,0.845309,0.0,0.0,2.535926,430.262046,1.183
472,060750332043,0.000000,144.694534,11.254019,0.000000,0.0,0.000000,8.038585,0.000000,0.000000,...,0.000000,0.0,0.535906,0.000000,2.143623,0.0,0.0,0.000000,224.008574,1.866
62,060750125012,1.631321,110.114192,105.220228,1.631321,0.0,0.815661,1.631321,0.815661,0.000000,...,0.815661,0.0,0.000000,1.631321,0.815661,0.0,0.0,0.000000,212.887439,1.226
269,060750229032,18.720749,34.321373,35.881435,1.560062,0.0,0.000000,6.240250,43.681747,4.680187,...,28.081123,0.0,6.240250,6.240250,6.240250,0.0,0.0,0.000000,188.767551,0.641
468,060750332031,0.360620,140.641904,10.457988,0.000000,0.0,0.000000,2.884962,0.000000,0.000000,...,0.000000,0.0,0.000000,0.721241,0.360620,0.0,0.0,0.360620,155.066715,2.773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,060759804011,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000
172,060750176011,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000
552,060750605021,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.966
553,060750605022,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.794


In [73]:
joint19.set_index('geoid').to_csv('clean_data/cum_ev_normalized.csv')